In [ ]:
from torch import nn
from collections import OrderedDict
import torch.nn.functional as F
import torch

In [ ]:
SUM = lambda x,y : x+y

In [ ]:
AVG = lambda x,y : (x+y)/2

In [ ]:
MAX = lambda x,y : torch.max(x,y)

In [ ]:
def check_equity(property,a,b):
    pa = getattr(a,property)
    pb = getattr(b,property)
    assert  pa==pb, "Different {}: {}!={}".format(property,pa,pb)

    return pa

In [ ]:
def tied_batchnorm2d(x: torch.Tensor, bn_a : nn.BatchNorm2d, bn_b : nn.BatchNorm2d,
                     running_mean:torch.Tensor, running_var:torch.Tensor,
                     training:bool,momentum,eps,op):

    return F.batch_norm(x,running_mean,running_var,
                        op(bn_a.weight,bn_b.weight),op(bn_a.bias,bn_b.bias),
                        training, momentum,eps)

In [ ]:
def tied_conv2d(x: torch.Tensor, conv_a : nn.Conv2d, conv_b : nn.Conv2d, op) -> torch.Tensor:
    parameters = dict()
    parameters['input'] = x
    parameters['weight'] = op(conv_a.weight, conv_b.weight)
    parameters['bias'] = op(conv_a.bias, conv_b.bias) if conv_a.bias is not None else None

    for p in ['stride','padding','dilation','groups']:
        parameters[p] = check_equity(p,conv_a,conv_b)

    return F.conv2d(**parameters)

In [ ]:
def module_unwrap(mod:nn.Module,recursive=False):
    children = OrderedDict()
    try:
        for name, module in mod.named_children():
            if (recursive):
                recursive_call = module_unwrap(module,recursive=True)
                if (len(recursive_call)>0):
                    children+=recursive_call
                else:
                    children[name] = module
            else:
                children[name] = module
    except AttributeError:
        pass

    return children

In [ ]:
class VGGBlock(nn.Module):
    def __init__(self, in_channels, out_channels,batch_norm=False):

        super().__init__()

        conv2_params = {'kernel_size': (3, 3),
                        'stride'     : (1, 1),
                        'padding'   : 1
                        }

        noop = lambda x : x

        self._batch_norm = batch_norm

        self.conv1 = nn.Conv2d(in_channels=in_channels,out_channels=out_channels , **conv2_params)
        self.bn1 = nn.BatchNorm2d(out_channels) if batch_norm else noop

        self.conv2 = nn.Conv2d(in_channels=out_channels,out_channels=out_channels, **conv2_params)
        self.bn2 = nn.BatchNorm2d(out_channels) if batch_norm else noop

        self.max_pooling = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

    @property
    def batch_norm(self):
        return self._batch_norm

    def forward(self,x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)

        x = self.max_pooling(x)

        return x

In [ ]:
class TiedVGGBlock(nn.Module):
    def __init__(self,block_a: VGGBlock,block_b:VGGBlock,operation=AVG):
        super().__init__()

        assert block_a.batch_norm == block_b.batch_norm

        self._block_a = module_unwrap(block_a)
        self._block_b = module_unwrap(block_b)
        self._operation = operation

        if block_a.batch_norm:
            assert block_a.conv1.out_channels == block_b.conv1.out_channels

            self.register_buffer('running_mean_1', torch.zeros(block_a.conv1.out_channels))
            self.register_buffer('running_var_1', torch.ones(block_a.conv1.out_channels))
            self.register_buffer('num_batches_tracked_1', torch.tensor(0, dtype=torch.long))

            assert block_a.conv2.out_channels == block_b.conv2.out_channels

            self.register_buffer('running_mean_2', torch.zeros(block_a.conv2.out_channels))
            self.register_buffer('running_var_2', torch.ones(block_a.conv2.out_channels))
            self.register_buffer('num_batches_tracked_2', torch.tensor(0, dtype=torch.long))
            

    def forward(self,x):


        training =  self._block_a['bn1'].training

        assert self._block_a['bn1'].training == self._block_b['bn2'].training
        assert self._block_a['bn1'].momentum == self._block_b['bn2'].momentum
        assert self._block_a['bn1'].eps == self._block_b['bn2'].eps
        assert self._block_a['max_pooling'].kernel_size ==  self._block_b['max_pooling'].kernel_size
        assert self._block_a['max_pooling'].stride == self._block_b['max_pooling'].stride

        if self._block_a['bn1'].momentum is None:
            exponential_average_factor = 0.0
        else:
            exponential_average_factor = self._block_a['bn1'].momentum

        if training:
            # TODO: if statement only here to tell the jit to skip emitting this when it is None
            if self.num_batches_tracked_1 is not None:
                self.num_batches_tracked_1 = self.num_batches_tracked_1 + 1
                self.num_batches_tracked_2 = self.num_batches_tracked_2 + 1
                if self._block_a['bn1'].momentum  is None:  # use cumulative moving average
                    exponential_average_factor = 1.0 / float(self.num_batches_tracked_1)
                else:  # use exponential moving average
                    exponential_average_factor = self._block_a['bn1'].momentum

        x = tied_conv2d(x, self._block_a['conv1'], self._block_b['conv1'], self._operation)
        x = tied_batchnorm2d(x,self._block_a['bn1'],self._block_b['bn1'],
                             self.running_mean_1,self.running_var_1, self._block_a['bn1'].training,
                             exponential_average_factor, self._block_a['bn1'].eps, self._operation)

        x = tied_conv2d(x, self._block_a['conv2'], self._block_b['conv2'], self._operation)
        x = tied_batchnorm2d(x, self._block_a['bn2'], self._block_b['bn2'],
                             self.running_mean_1, self.running_var_1, self._block_a['bn2'].training,
                             exponential_average_factor, self._block_a['bn2'].eps, self._operation)

        x = F.max_pool2d(x,kernel_size=self._block_a['max_pooling'].kernel_size,stride=self._block_a['max_pooling'].stride)
        return x

In [ ]:
class VGG16(nn.Module):

  def __init__(self, input_size, num_classes=1,classifier = None,batch_norm=False):
    super(VGG16, self).__init__()

    self.in_channels,self.in_width,self.in_height = input_size

    self.block_1 = VGGBlock(self.in_channels,64,batch_norm=batch_norm)
    self.block_2 = VGGBlock(64, 128,batch_norm=batch_norm)
    self.block_3 = VGGBlock(128, 256,batch_norm=batch_norm)
    self.block_4 = VGGBlock(256,512,batch_norm=batch_norm)

    self.classifier = classifier

    if (self.classifier is None):

        self.classifier = nn.Sequential(
          nn.Linear(2048, 2048),
          nn.ReLU(True),
          nn.Dropout(p=0.5),
          nn.Linear(2048, 512),
          nn.ReLU(True),
          nn.Dropout(p=0.5),
          nn.Linear(512, num_classes)
        )


  @property
  def input_size(self):
      return self.in_channels,self.in_width,self.in_height

  def forward(self, x):

    x = self.block_1(x)
    x = self.block_2(x)
    x = self.block_3(x)
    x = self.block_4(x)
    # x = self.avgpool(x)
    x = torch.flatten(x,1)

    x = self.classifier(x)
    return x

In [ ]:
class TiedVGG16(nn.Module):

  def __init__(self, vgg_a,vgg_b,classifier_from=0,operation=SUM):
    super().__init__()

    assert 0 <= classifier_from <= 1

    assert vgg_a.input_size == vgg_b.input_size

    self.in_channels, self.in_width, self.in_height = vgg_a.input_size

    self._vgg_a = vgg_a
    self._vgg_b = vgg_b
    self._operation = operation

    self.tblock_1 = TiedVGGBlock(vgg_a.block_1, vgg_b.block_1, operation)
    self.tblock_2 = TiedVGGBlock(vgg_a.block_2, vgg_b.block_2, operation)
    self.tblock_3 = TiedVGGBlock(vgg_a.block_3, vgg_b.block_3, operation)
    self.tblock_4 = TiedVGGBlock(vgg_a.block_4, vgg_b.block_4, operation)

    self.classifier = vgg_a.classifier if classifier_from == 0 else vgg_b.classifier


  def forward(self, x1,x2):

    o1 = self._vgg_a(x1)
    o2 = self._vgg_b(x2)
    #
    xc = torch.cat((x1,x2),0)

    xc = self.tblock_1(xc)
    xc = self.tblock_2(xc)
    xc = self.tblock_3(xc)
    xc = self.tblock_4(xc)
    xc = torch.flatten(xc,1)
    xc = self.classifier(xc)

    return [o1,o2,xc]

In [ ]:
class CombinedLoss(nn.Module):
    def __init__(self, loss_a, loss_b, loss_combo, _lambda=1.0):
        super().__init__()
        self.loss_a = loss_a
        self.loss_b = loss_b
        self.loss_combo = loss_combo

        self.register_buffer('_lambda',torch.tensor(float(_lambda),dtype=torch.float32))


    def forward(self,y_hat,y):

        return self.loss_a(y_hat[0],y[0]) + self.loss_b(y_hat[1],y[1]) + self._lambda * self.loss_combo(y_hat[2],torch.cat(y,0))

In [ ]:
import torch.nn.functional as F
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision
import random
from torch.utils.data import Subset
from matplotlib import pyplot as plt
from torchsummary import summary
from torchvision import transforms
import progressbar as pb
import numpy as np

In [ ]:
random.seed(47)

In [ ]:
def unwrap_model(model):
  for s,m in model.named_modules():
    print (s)

TRAINING FUNCTION CHE SI BLOCCA DOPO CIRCA 1 MINUTO: INDEX OUT OF RANGE

In [ ]:
def train(net, loaders, optimizer, criterion, epochs=20, dev=None, save_param=False, model_name="valerio"):
      loaders_a, loaders_b = loaders
    # try:
      net = net.to(dev)
      #print(net)
      #summary(net,[(net.in_channels,net.in_width,net.in_height)]*2)


      criterion.to(dev)


      # Initialize history
      history_loss = {"train": [], "val": [], "test": []}
      history_accuracy_combo = {"train": [], "val": [], "test": []}
      history_accuracy_a = {"train": [], "val": [], "test": []}
      history_accuracy_b = {"train": [], "val": [], "test": []}
      # Store the best val accuracy
      best_val_accuracy = 0

      # Process each epoch
      for epoch in range(epochs):
        # Initialize epoch variables
        sum_loss = {"train": 0, "val": 0, "test": 0}
        sum_accuracy_combo = {"train": 0, "val": 0, "test": 0}
        sum_accuracy_a = {"train": 0, "val": 0, "test": 0}
        sum_accuracy_b = {"train": 0, "val": 0, "test": 0}

        progbar = None
        # Process each split
        for split in ["train", "val", "test"]:
          if split == "train":
            net.train()
            #widgets = [
              #' [', pb.Timer(), '] ',
              #pb.Bar(),
              #' [', pb.ETA(), '] ', pb.Variable('ta','[Train Acc: {formatted_value}]')]

            #progbar = pb.ProgressBar(max_value=len(loaders_a[split]),widgets=widgets,redirect_stdout=True)

          else:
            net.eval()
          # Process each batch
          for j, ((input_a, labels_a), (input_b, labels_b)) in enumerate(zip(loaders_a[split], loaders_b[split])):
            labels_a = labels_a.unsqueeze(1).float()
            labels_b = labels_b.unsqueeze(1).float()

            input_a = input_a.to(dev)
            labels_a = labels_a.to(dev)
            input_b = input_b.to(dev)
            labels_b = labels_b.to(dev)

            # Reset gradients
            optimizer.zero_grad()
            # Compute output
            pred = net(input_a,input_b)

            loss = criterion(pred, [labels_a, labels_b])
            # Update loss
            sum_loss[split] += loss.item()
            # Check parameter update
            if split == "train":
              # Compute gradients
              loss.backward()
              # Optimize
              optimizer.step()

            # Compute accuracy
            pred_labels = (pred[2] >= 0.0).long()  # Binarize predictions to 0 and 1
            pred_labels_a = (pred[0] >= 0.0).long()  # Binarize predictions to 0 and 1
            pred_labels_b = (pred[1] >= 0.0).long()  # Binarize predictions to 0 and 1


            labels = torch.cat((labels_a, labels_b), 0)
            batch_accuracy_combo = (pred_labels == labels).sum().item() / len(labels)
            batch_accuracy_a = (pred_labels_a == labels_a).sum().item() / len(labels_a)
            batch_accuracy_b = (pred_labels_b == labels_b).sum().item() / len(labels_b)
            # Update accuracy
            sum_accuracy_combo[split] += batch_accuracy_combo
            sum_accuracy_a[split] += batch_accuracy_a
            sum_accuracy_b[split] += batch_accuracy_b

            #if (split=='train'):
              #progbar.update(j, ta=batch_accuracy)
              #progbar.update(j, ta=batch_accuracy_a)
              #progbar.update(j, ta=batch_accuracy_b)

        #if (progbar is not None):
          #progbar.finish()
        # Compute epoch loss/accuracy
        #for split in ["train", "val", "test"]:
          #epoch_loss = sum_loss[split] / (len(loaders_a[split])+len(loaders_b[split])) 
          #epoch_accuracy_combo = {split: sum_accuracy_combo[split] / len(loaders[split]) for split in ["train", "val", "test"]}
          #epoch_accuracy_a = sum_accuracy_a[split] / len(loaders_a[split])
          #epoch_accuracy_b = sum_accuracy_b[split] / len(loaders_b[split])
        epoch_loss = sum_loss["train"] / (len(loaders_a["train"])+len(loaders_b["train"])) 
        epoch_accuracy_a = sum_accuracy_a["train"] / len(loaders_a["train"])
        epoch_accuracy_b = sum_accuracy_b["train"] / len(loaders_b["train"])
        epoch_accuracy_combo = sum_accuracy_combo["train"] / len(loaders_a["train"]) 

        epoch_loss_val = sum_loss["val"] / (len(loaders_a["val"])+len(loaders_b["val"])) 
        epoch_accuracy_a_val = sum_accuracy_a["val"] / len(loaders_a["val"])
        epoch_accuracy_b_val = sum_accuracy_b["val"] / len(loaders_b["val"])
        epoch_accuracy_combo_val = sum_accuracy_combo["val"] / len(loaders_a["val"]) 

        epoch_loss_test = sum_loss["test"] / (len(loaders_a["test"])+len(loaders_b["test"])) 
        epoch_accuracy_a_test = sum_accuracy_a["test"] / len(loaders_a["test"])
        epoch_accuracy_b_test = sum_accuracy_b["test"] / len(loaders_b["test"])
        epoch_accuracy_combo_test = sum_accuracy_combo["test"] / len(loaders_a["test"]) 


        # Store params at the best validation accuracy
        if save_param and epoch_accuracy["val"] > best_val_accuracy:
          # torch.save(net.state_dict(), f"{net.__class__.__name__}_best_val.pth")
          torch.save(net.state_dict(), f"{model_name}_best_val.pth")
          best_val_accuracy = epoch_accuracy["val"]

        # Update history
        for split in ["train", "val", "test"]:
          history_loss[split].append(epoch_loss)
          history_accuracy_a[split].append(epoch_accuracy_a)
          history_accuracy_b[split].append(epoch_accuracy_b)
        # Print info
        print(f"Epoch {epoch + 1}:",
              f"Training Loss for combo = {epoch_loss:.4f},",)
        print(f"Epoch {epoch + 1}:",
              f"Training Accuracy for A = {epoch_accuracy_a:.4f},")
        print(f"Epoch {epoch + 1}:",
              f"Training Accuracy for B = {epoch_accuracy_b:.4f},")
        print(f"Epoch {epoch + 1}:",
              f"Training Accuracy for combo = {epoch_accuracy_combo:.4f},")
        
        print(f"Epoch {epoch + 1}:",
              f"Val Loss for combo = {epoch_loss_val:.4f},",)
        print(f"Epoch {epoch + 1}:",
              f"Val Accuracy for A = {epoch_accuracy_a_val:.4f},")
        print(f"Epoch {epoch + 1}:",
              f"Val Accuracy for B = {epoch_accuracy_b_val:.4f},")
        print(f"Epoch {epoch + 1}:",
              f"Val Accuracy for combo = {epoch_accuracy_combo_val:.4f},")
        
        print(f"Epoch {epoch + 1}:",
              f"Test Loss for combo = {epoch_loss_test:.4f},",)
        print(f"Epoch {epoch + 1}:",
              f"Test Accuracy for A = {epoch_accuracy_a_test:.4f},")
        print(f"Epoch {epoch + 1}:",
              f"Test Accuracy for B = {epoch_accuracy_b_test:.4f},")
        print(f"Epoch {epoch + 1}:",
              f"Test Accuracy for combo = {epoch_accuracy_combo_test:.4f},")
        print("\n")

In [ ]:
def parse_dataset(dataset):

  dataset.targets = dataset.targets % 2

  return dataset

In [ ]:
root_dir = './'

In [ ]:
rescale_data = transforms.Lambda(lambda x : x/255)

In [ ]:
# Compose transformations
data_transform = transforms.Compose([
  transforms.Resize(32),
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
  rescale_data,
])

test_transform = transforms.Compose([
  transforms.Resize(32),
  transforms.ToTensor(),
  rescale_data,
])
# Load MNIST dataset with transforms
train_set = torchvision.datasets.MNIST(root=root_dir, train=True, download=True, transform=data_transform)
test_set = torchvision.datasets.MNIST(root=root_dir, train=False, download=True, transform=test_transform)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
train_set = parse_dataset(train_set)
test_set = parse_dataset(test_set)

In [ ]:
# Dataset len
num_train = len(train_set)
num_test = len(test_set)
print(f"Num. training samples: {num_train}")
print(f"Num. test samples:     {num_test}")

Num. training samples: 60000
Num. test samples:     10000


In [ ]:
# List of indexes on the training set
train_idx = list(range(num_train))

# List of indexes of the test set
test_idx = list(range(num_test))

# Shuffle the training set

random.shuffle(train_idx)

In [ ]:
val_frac = 0.1

# Number of samples of the validation set
num_val = int(num_train * val_frac)
num_train = num_train - num_val

print(f"{num_train} samples used as train set")
print(f"{num_val}  samples used as val set")

idx = range(0,len(train_set))
h = len(idx)//2

set_a_idx = idx[:h]
set_b_idx = idx[h:]

h_val = int(h*val_frac)

train_idx_a = set_a_idx[:h_val]
val_idx_a = set_a_idx[h_val:]

train_idx_b = set_b_idx[:h_val]
val_idx_b = set_b_idx[h_val:]

val_set_a = Subset(train_set, val_idx_a)
val_set_b = Subset(train_set, val_idx_b)
train_set_a = Subset(train_set, train_idx_a)
train_set_b = Subset(train_set, train_idx_b)

num_test = int(len(test_set)/2)
# List of indexes of the test set
test_idx = list(range(num_test*2))

# Split test set
test_a_idx = test_idx[num_test:]
test_b_idx = test_idx[:num_test]

test_set_a = Subset(test_set, test_a_idx)
test_set_b = Subset(test_set, test_b_idx)

print(f"{num_train} samples used as train set")
print(f"{num_val} samples used as val set")
print(f"{num_test}  samples used as test set")
print("\n")
num_train_a1 = len(train_set_a)
num_train_b1 = len(train_set_b)
num_val_a = len(val_set_a)
num_val_b = len(val_set_b)
print(f"{num_train_a1} samples used as train set a")
print(f"{num_val_a}  samples used as val set a")
print(f"{num_train_b1} samples used as train set b")
print(f"{num_val_b}  samples used as val set b")

54000 samples used as train set
6000  samples used as val set
54000 samples used as train set
6000 samples used as val set
5000  samples used as test set


3000 samples used as train set a
27000  samples used as val set a
3000 samples used as train set b
27000  samples used as val set b


In [ ]:
#check
list(set(train_set_b.indices) & set(val_set_a.indices))

[]

In [ ]:
class ConcatDataset(torch.utils.data.Dataset):
    def __init__(self, *datasets):
        self.datasets = datasets

    def __getitem__(self, i):
        return tuple(d[i] for d in self.datasets)

    def __len__(self):
        return min(len(d) for d in self.datasets)

In [ ]:
# Define loaders

train_loader_a = DataLoader(train_set_a, batch_size=128, num_workers=0, shuffle=True, drop_last=True)
val_loader_a   = DataLoader(val_set_a,   batch_size=128, num_workers=0, shuffle=False, drop_last=False)
test_loader_a  = DataLoader(test_set_a,  batch_size=128, num_workers=0, shuffle=False, drop_last=False)

train_loader_b = DataLoader(train_set_b, batch_size=128, num_workers=0, shuffle=True, drop_last=True)
val_loader_b   = DataLoader(val_set_b,   batch_size=128, num_workers=0, shuffle=False, drop_last=False)
test_loader_b  = DataLoader(test_set_b,  batch_size=128, num_workers=0, shuffle=False, drop_last=False)

In [ ]:
# Define loaders

train_loader = DataLoader(ConcatDataset(train_set_a, train_set_b), batch_size=128, num_workers=0, shuffle=True, drop_last=True)
val_loader   = DataLoader(ConcatDataset(val_set_a, val_set_b),   batch_size=128, num_workers=0, shuffle=False, drop_last=False)
test_loader  = DataLoader(ConcatDataset(test_set_a, test_set_b),  batch_size=128, num_workers=0, shuffle=False, drop_last=False)

In [ ]:
# Define dictionary of loaders
loaders = {"train": train_loader,
           "val": val_loader,
           "test": test_loader}

In [ ]:
# Define dictionary of loaders
loaders_a = {"train": train_loader_a,
           "val": val_loader_a,
           "test": test_loader_a}

In [ ]:
loaders_b = {"train": train_loader_b,
           "val": val_loader_b,
           "test": test_loader_b}

In [ ]:
#model1 = VGG16((1,32,32),batch_norm=True)
#model2 = VGG16((1,32,32),batch_norm=True,classifier=model1.classifier)
#combo = TiedVGG16(model1,model2,0)

In [ ]:
model1 = VGG16((1,32,32),batch_norm=True)
model2 = VGG16((1,32,32),batch_norm=True,classifier=model1.classifier)
combo = TiedVGG16(model1,model2,0, AVG)

In [ ]:
dev = torch.device('cuda')

In [ ]:
optimizer = torch.optim.SGD(combo.parameters(), lr = 0.01)
# Define a loss
criterion = CombinedLoss(nn.BCEWithLogitsLoss(),nn.BCEWithLogitsLoss(),nn.BCEWithLogitsLoss(),_lambda = 1)
n_params = 0

In [ ]:
for n,p in combo.named_parameters():
   print("{:30} {:>20} {:>15}".format(n,str(np.asarray(p.shape)),p.numel()))
   n_params += p.numel()

_vgg_a.block_1.conv1.weight           [64  1  3  3]             576
_vgg_a.block_1.conv1.bias                      [64]              64
_vgg_a.block_1.bn1.weight                      [64]              64
_vgg_a.block_1.bn1.bias                        [64]              64
_vgg_a.block_1.conv2.weight           [64 64  3  3]           36864
_vgg_a.block_1.conv2.bias                      [64]              64
_vgg_a.block_1.bn2.weight                      [64]              64
_vgg_a.block_1.bn2.bias                        [64]              64
_vgg_a.block_2.conv1.weight       [128  64   3   3]           73728
_vgg_a.block_2.conv1.bias                     [128]             128
_vgg_a.block_2.bn1.weight                     [128]             128
_vgg_a.block_2.bn1.bias                       [128]             128
_vgg_a.block_2.conv2.weight       [128 128   3   3]          147456
_vgg_a.block_2.conv2.bias                     [128]             128
_vgg_a.block_2.bn2.weight                     [1

In [ ]:
print(f'Total number of parameters in combo model: {n_params:,}')

Total number of parameters in combo model: 14,622,081


In [ ]:
!pip install --upgrade progressbar2

  Found existing installation: progressbar2 3.38.0
    Uninstalling progressbar2-3.38.0:
      Successfully uninstalled progressbar2-3.38.0


In [ ]:
# Train model
train(combo, (loaders_a, loaders_b), optimizer, criterion, epochs=5, dev=dev)

Epoch 1: Training Loss for combo = 0.9425,
Epoch 1: Training Accuracy for A = 0.6637,
Epoch 1: Training Accuracy for B = 0.6481,
Epoch 1: Training Accuracy for combo = 0.6878,
Epoch 1: Val Loss for combo = 1.0702,
Epoch 1: Val Accuracy for A = 0.5108,
Epoch 1: Val Accuracy for B = 0.5054,
Epoch 1: Val Accuracy for combo = 0.5081,
Epoch 1: Test Loss for combo = 1.0718,
Epoch 1: Test Accuracy for A = 0.5029,
Epoch 1: Test Accuracy for B = 0.5086,
Epoch 1: Test Accuracy for combo = 0.5058,


Epoch 2: Training Loss for combo = 0.5821,
Epoch 2: Training Accuracy for A = 0.8499,
Epoch 2: Training Accuracy for B = 0.8465,
Epoch 2: Training Accuracy for combo = 0.8448,
Epoch 2: Val Loss for combo = 17.1384,
Epoch 2: Val Accuracy for A = 0.5108,
Epoch 2: Val Accuracy for B = 0.5054,
Epoch 2: Val Accuracy for combo = 0.5081,
Epoch 2: Test Loss for combo = 17.2232,
Epoch 2: Test Accuracy for A = 0.5029,
Epoch 2: Test Accuracy for B = 0.5086,
Epoch 2: Test Accuracy for combo = 0.5058,


Epoch 3: T

CONTROLLARE TRAINING. PROVARE LAMBDA (SEMPRE POSITIVO, MAGARI TRA 0 E 1). PROVARE SIA SUM, AVG, MAX. FARE TABELLA CON I RISULTATI. PROVARE CAMBIARE OPTIMIZER, LEARNING RATE. AGGIUNGERE ACCURACY MODELLO COMBINATO.

CASO SUM: TEST CON DIVERSI OPTIMIZER, LEARNING RATE E LAMBDA

learning rate = 0.001, SGD e lambda=0.5

In [ ]:
model1 = VGG16((1,32,32),batch_norm=True)
model2 = VGG16((1,32,32),batch_norm=True,classifier=model1.classifier)
combo = TiedVGG16(model1,model2,0, SUM)

In [ ]:
optimizer = torch.optim.SGD(combo.parameters(), lr = 0.001)
# Define a loss
criterion = CombinedLoss(nn.BCEWithLogitsLoss(),nn.BCEWithLogitsLoss(),nn.BCEWithLogitsLoss(),_lambda = 0.5)
n_params = 0

In [ ]:
# Train model
train(combo, (loaders_a, loaders_b), optimizer, criterion, epochs=5, dev=dev)

Epoch 1: Training Loss for combo = 0.8681,
Epoch 1: Training Accuracy for A = 0.5594,
Epoch 1: Training Accuracy for B = 0.5272,
Epoch 1: Training Accuracy for combo = 0.5132,
Epoch 1: Val Loss for combo = 0.9150,
Epoch 1: Val Accuracy for A = 0.5108,
Epoch 1: Val Accuracy for B = 0.5054,
Epoch 1: Val Accuracy for combo = 0.5081,
Epoch 1: Test Loss for combo = 0.9160,
Epoch 1: Test Accuracy for A = 0.5029,
Epoch 1: Test Accuracy for B = 0.5086,
Epoch 1: Test Accuracy for combo = 0.5058,


Epoch 2: Training Loss for combo = 0.8482,
Epoch 2: Training Accuracy for A = 0.5768,
Epoch 2: Training Accuracy for B = 0.5768,
Epoch 2: Training Accuracy for combo = 0.5831,
Epoch 2: Val Loss for combo = 0.9080,
Epoch 2: Val Accuracy for A = 0.5108,
Epoch 2: Val Accuracy for B = 0.5054,
Epoch 2: Val Accuracy for combo = 0.5081,
Epoch 2: Test Loss for combo = 0.9091,
Epoch 2: Test Accuracy for A = 0.5029,
Epoch 2: Test Accuracy for B = 0.5086,
Epoch 2: Test Accuracy for combo = 0.5058,


Epoch 3: Tra

learning rate = 0.0001, Adam e lambda=0.7 e 7 epoche



In [ ]:
model1 = VGG16((1,32,32),batch_norm=True)
model2 = VGG16((1,32,32),batch_norm=True,classifier=model1.classifier)
combo = TiedVGG16(model1,model2,0, SUM)

In [ ]:
optimizer = torch.optim.Adam(combo.parameters(), lr = 0.0001)
# Define a loss
criterion = CombinedLoss(nn.BCEWithLogitsLoss(),nn.BCEWithLogitsLoss(),nn.BCEWithLogitsLoss(),_lambda = 0.7)
n_params = 0

In [ ]:
# Train model
train(combo, (loaders_a, loaders_b), optimizer, criterion, epochs=7, dev=dev)

Epoch 1: Training Loss for combo = 0.5693,
Epoch 1: Training Accuracy for A = 0.7884,
Epoch 1: Training Accuracy for B = 0.8013,
Epoch 1: Training Accuracy for combo = 0.8145,
Epoch 1: Val Loss for combo = 5.5858,
Epoch 1: Val Accuracy for A = 0.5108,
Epoch 1: Val Accuracy for B = 0.5054,
Epoch 1: Val Accuracy for combo = 0.5081,
Epoch 1: Test Loss for combo = 5.6102,
Epoch 1: Test Accuracy for A = 0.5029,
Epoch 1: Test Accuracy for B = 0.5086,
Epoch 1: Test Accuracy for combo = 0.5058,


Epoch 2: Training Loss for combo = 0.2154,
Epoch 2: Training Accuracy for A = 0.9443,
Epoch 2: Training Accuracy for B = 0.9406,
Epoch 2: Training Accuracy for combo = 0.9360,
Epoch 2: Val Loss for combo = 8.6365,
Epoch 2: Val Accuracy for A = 0.5108,
Epoch 2: Val Accuracy for B = 0.5054,
Epoch 2: Val Accuracy for combo = 0.5081,
Epoch 2: Test Loss for combo = 8.6754,
Epoch 2: Test Accuracy for A = 0.5029,
Epoch 2: Test Accuracy for B = 0.5086,
Epoch 2: Test Accuracy for combo = 0.5058,


Epoch 3: Tra

learning rate = 0.001, Adam e lambda=0.2 e 7 epoche



In [ ]:
model1 = VGG16((1,32,32),batch_norm=True)
model2 = VGG16((1,32,32),batch_norm=True,classifier=model1.classifier)
combo = TiedVGG16(model1,model2,0, SUM)

In [ ]:
optimizer = torch.optim.Adam(combo.parameters(), lr = 0.001)
# Define a loss
criterion = CombinedLoss(nn.BCEWithLogitsLoss(),nn.BCEWithLogitsLoss(),nn.BCEWithLogitsLoss(),_lambda = 0.2)
n_params = 0

In [ ]:
# Train model
train(combo, (loaders_a, loaders_b), optimizer, criterion, epochs=7, dev=dev)

Epoch 1: Training Loss for combo = 0.9494,
Epoch 1: Training Accuracy for A = 0.6046,
Epoch 1: Training Accuracy for B = 0.6253,
Epoch 1: Training Accuracy for combo = 0.6262,
Epoch 1: Val Loss for combo = 1.0819,
Epoch 1: Val Accuracy for A = 0.5108,
Epoch 1: Val Accuracy for B = 0.5054,
Epoch 1: Val Accuracy for combo = 0.4919,
Epoch 1: Test Loss for combo = 1.0837,
Epoch 1: Test Accuracy for A = 0.5029,
Epoch 1: Test Accuracy for B = 0.5086,
Epoch 1: Test Accuracy for combo = 0.4942,


Epoch 2: Training Loss for combo = 0.3252,
Epoch 2: Training Accuracy for A = 0.8978,
Epoch 2: Training Accuracy for B = 0.8838,
Epoch 2: Training Accuracy for combo = 0.8663,
Epoch 2: Val Loss for combo = 3.0149,
Epoch 2: Val Accuracy for A = 0.5108,
Epoch 2: Val Accuracy for B = 0.5054,
Epoch 2: Val Accuracy for combo = 0.4919,
Epoch 2: Test Loss for combo = 3.0238,
Epoch 2: Test Accuracy for A = 0.5029,
Epoch 2: Test Accuracy for B = 0.5086,
Epoch 2: Test Accuracy for combo = 0.4942,


Epoch 3: Tra

CASO AVG: TEST CON DIVERSI OPTIMIZER, LEARNING RATE E LAMBDA

learning rate = 0.001, SGD e lambda=0.5

In [ ]:
model1 = VGG16((1,32,32),batch_norm=True)
model2 = VGG16((1,32,32),batch_norm=True,classifier=model1.classifier)
combo = TiedVGG16(model1,model2,0, AVG)

In [ ]:
optimizer = torch.optim.SGD(combo.parameters(), lr = 0.001)
# Define a loss
criterion = CombinedLoss(nn.BCEWithLogitsLoss(),nn.BCEWithLogitsLoss(),nn.BCEWithLogitsLoss(),_lambda = 0.5)
n_params = 0

In [ ]:
# Train model
train(combo, (loaders_a, loaders_b), optimizer, criterion, epochs=5, dev=dev)

Epoch 1: Training Loss for combo = 0.8681,
Epoch 1: Training Accuracy for A = 0.5082,
Epoch 1: Training Accuracy for B = 0.5377,
Epoch 1: Training Accuracy for combo = 0.5105,
Epoch 1: Val Loss for combo = 0.8710,
Epoch 1: Val Accuracy for A = 0.5108,
Epoch 1: Val Accuracy for B = 0.5054,
Epoch 1: Val Accuracy for combo = 0.5081,
Epoch 1: Test Loss for combo = 0.8713,
Epoch 1: Test Accuracy for A = 0.5029,
Epoch 1: Test Accuracy for B = 0.5086,
Epoch 1: Test Accuracy for combo = 0.5058,


Epoch 2: Training Loss for combo = 0.8540,
Epoch 2: Training Accuracy for A = 0.5554,
Epoch 2: Training Accuracy for B = 0.5635,
Epoch 2: Training Accuracy for combo = 0.5520,
Epoch 2: Val Loss for combo = 1.1069,
Epoch 2: Val Accuracy for A = 0.5108,
Epoch 2: Val Accuracy for B = 0.5054,
Epoch 2: Val Accuracy for combo = 0.5081,
Epoch 2: Test Loss for combo = 1.1092,
Epoch 2: Test Accuracy for A = 0.5029,
Epoch 2: Test Accuracy for B = 0.5086,
Epoch 2: Test Accuracy for combo = 0.5058,


Epoch 3: Tra

learning rate = 0.0001, Adam e lambda=0.7 e 7 epoche



In [ ]:
model1 = VGG16((1,32,32),batch_norm=True)
model2 = VGG16((1,32,32),batch_norm=True,classifier=model1.classifier)
combo = TiedVGG16(model1,model2,0, AVG)

In [ ]:
optimizer = torch.optim.Adam(combo.parameters(), lr = 0.0001)
# Define a loss
criterion = CombinedLoss(nn.BCEWithLogitsLoss(),nn.BCEWithLogitsLoss(),nn.BCEWithLogitsLoss(),_lambda = 0.7)
n_params = 0

In [ ]:
# Train model
train(combo, (loaders_a, loaders_b), optimizer, criterion, epochs=7, dev=dev)

Epoch 1: Training Loss for combo = 0.5401,
Epoch 1: Training Accuracy for A = 0.8149,
Epoch 1: Training Accuracy for B = 0.7993,
Epoch 1: Training Accuracy for combo = 0.8050,
Epoch 1: Val Loss for combo = 2.9940,
Epoch 1: Val Accuracy for A = 0.5108,
Epoch 1: Val Accuracy for B = 0.5054,
Epoch 1: Val Accuracy for combo = 0.4919,
Epoch 1: Test Loss for combo = 2.9929,
Epoch 1: Test Accuracy for A = 0.5029,
Epoch 1: Test Accuracy for B = 0.5086,
Epoch 1: Test Accuracy for combo = 0.4942,


Epoch 2: Training Loss for combo = 0.2115,
Epoch 2: Training Accuracy for A = 0.9429,
Epoch 2: Training Accuracy for B = 0.9446,
Epoch 2: Training Accuracy for combo = 0.9348,
Epoch 2: Val Loss for combo = 23.1160,
Epoch 2: Val Accuracy for A = 0.5108,
Epoch 2: Val Accuracy for B = 0.5054,
Epoch 2: Val Accuracy for combo = 0.4919,
Epoch 2: Test Loss for combo = 23.0310,
Epoch 2: Test Accuracy for A = 0.5029,
Epoch 2: Test Accuracy for B = 0.5086,
Epoch 2: Test Accuracy for combo = 0.4942,


Epoch 3: T


learning rate = 0.001, Adam e lambda=0.2 e 7 epoche



In [ ]:
model1 = VGG16((1,32,32),batch_norm=True)
model2 = VGG16((1,32,32),batch_norm=True,classifier=model1.classifier)
combo = TiedVGG16(model1,model2,0, AVG)

In [ ]:
optimizer = torch.optim.Adam(combo.parameters(), lr = 0.001)
# Define a loss
criterion = CombinedLoss(nn.BCEWithLogitsLoss(),nn.BCEWithLogitsLoss(),nn.BCEWithLogitsLoss(),_lambda = 0.2)
n_params = 0

In [ ]:
# Train model
train(combo, (loaders_a, loaders_b), optimizer, criterion, epochs=7, dev=dev)

Epoch 1: Training Loss for combo = 1.0287,
Epoch 1: Training Accuracy for A = 0.4983,
Epoch 1: Training Accuracy for B = 0.5088,
Epoch 1: Training Accuracy for combo = 0.5058,
Epoch 1: Val Loss for combo = 0.8008,
Epoch 1: Val Accuracy for A = 0.4892,
Epoch 1: Val Accuracy for B = 0.5054,
Epoch 1: Val Accuracy for combo = 0.4919,
Epoch 1: Test Loss for combo = 0.8001,
Epoch 1: Test Accuracy for A = 0.4971,
Epoch 1: Test Accuracy for B = 0.5086,
Epoch 1: Test Accuracy for combo = 0.4942,


Epoch 2: Training Loss for combo = 0.5886,
Epoch 2: Training Accuracy for A = 0.7174,
Epoch 2: Training Accuracy for B = 0.7592,
Epoch 2: Training Accuracy for combo = 0.7250,
Epoch 2: Val Loss for combo = 2.5800,
Epoch 2: Val Accuracy for A = 0.5108,
Epoch 2: Val Accuracy for B = 0.5054,
Epoch 2: Val Accuracy for combo = 0.4919,
Epoch 2: Test Loss for combo = 2.5862,
Epoch 2: Test Accuracy for A = 0.5029,
Epoch 2: Test Accuracy for B = 0.5086,
Epoch 2: Test Accuracy for combo = 0.4942,


Epoch 3: Tra

CASO MAX: TEST CON DIVERSI OPTIMIZER, LEARNING RATE E LAMBDA

In [ ]:
model1 = VGG16((1,32,32),batch_norm=True)
model2 = VGG16((1,32,32),batch_norm=True,classifier=model1.classifier)
combo = TiedVGG16(model1,model2,0, MAX)

In [ ]:
optimizer = torch.optim.SGD(combo.parameters(), lr = 0.01)
# Define a loss
criterion = CombinedLoss(nn.BCEWithLogitsLoss(),nn.BCEWithLogitsLoss(),nn.BCEWithLogitsLoss(),_lambda = 1)
n_params = 0

In [ ]:
# Train model
train(combo, (loaders_a, loaders_b), optimizer, criterion, epochs=5, dev=dev)

Epoch 1: Training Loss for combo = 0.9775,
Epoch 1: Training Accuracy for A = 0.6467,
Epoch 1: Training Accuracy for B = 0.6332,
Epoch 1: Training Accuracy for combo = 0.6068,
Epoch 1: Val Loss for combo = 1.0919,
Epoch 1: Val Accuracy for A = 0.5108,
Epoch 1: Val Accuracy for B = 0.5054,
Epoch 1: Val Accuracy for combo = 0.5081,
Epoch 1: Test Loss for combo = 1.0933,
Epoch 1: Test Accuracy for A = 0.5029,
Epoch 1: Test Accuracy for B = 0.5086,
Epoch 1: Test Accuracy for combo = 0.5058,


Epoch 2: Training Loss for combo = 0.7850,
Epoch 2: Training Accuracy for A = 0.8200,
Epoch 2: Training Accuracy for B = 0.7986,
Epoch 2: Training Accuracy for combo = 0.6233,
Epoch 2: Val Loss for combo = 1.3326,
Epoch 2: Val Accuracy for A = 0.5108,
Epoch 2: Val Accuracy for B = 0.5054,
Epoch 2: Val Accuracy for combo = 0.5081,
Epoch 2: Test Loss for combo = 1.3369,
Epoch 2: Test Accuracy for A = 0.5029,
Epoch 2: Test Accuracy for B = 0.5086,
Epoch 2: Test Accuracy for combo = 0.5058,


Epoch 3: Tra

learning rate = 0.001, SGD e lambda=0.5

In [ ]:
model1 = VGG16((1,32,32),batch_norm=True)
model2 = VGG16((1,32,32),batch_norm=True,classifier=model1.classifier)
combo = TiedVGG16(model1,model2,0, MAX)

In [ ]:
optimizer = torch.optim.SGD(combo.parameters(), lr = 0.001)
# Define a loss
criterion = CombinedLoss(nn.BCEWithLogitsLoss(),nn.BCEWithLogitsLoss(),nn.BCEWithLogitsLoss(),_lambda = 0.5)
n_params = 0

In [ ]:
# Train model
train(combo, (loaders_a, loaders_b), optimizer, criterion, epochs=5, dev=dev)

Epoch 1: Training Loss for combo = 0.8627,
Epoch 1: Training Accuracy for A = 0.5360,
Epoch 1: Training Accuracy for B = 0.5391,
Epoch 1: Training Accuracy for combo = 0.5328,
Epoch 1: Val Loss for combo = 0.8676,
Epoch 1: Val Accuracy for A = 0.4892,
Epoch 1: Val Accuracy for B = 0.4946,
Epoch 1: Val Accuracy for combo = 0.5081,
Epoch 1: Test Loss for combo = 0.8677,
Epoch 1: Test Accuracy for A = 0.4971,
Epoch 1: Test Accuracy for B = 0.4914,
Epoch 1: Test Accuracy for combo = 0.5058,


Epoch 2: Training Loss for combo = 0.8540,
Epoch 2: Training Accuracy for A = 0.5533,
Epoch 2: Training Accuracy for B = 0.5618,
Epoch 2: Training Accuracy for combo = 0.5671,
Epoch 2: Val Loss for combo = 0.8677,
Epoch 2: Val Accuracy for A = 0.4892,
Epoch 2: Val Accuracy for B = 0.4946,
Epoch 2: Val Accuracy for combo = 0.5081,
Epoch 2: Test Loss for combo = 0.8678,
Epoch 2: Test Accuracy for A = 0.4971,
Epoch 2: Test Accuracy for B = 0.4914,
Epoch 2: Test Accuracy for combo = 0.5058,


Epoch 3: Tra

learning rate = 0.0001, Adam e lambda=0.7 e 7 epoche



In [ ]:
model1 = VGG16((1,32,32),batch_norm=True)
model2 = VGG16((1,32,32),batch_norm=True,classifier=model1.classifier)
combo = TiedVGG16(model1,model2,0, MAX)

In [ ]:
optimizer = torch.optim.Adam(combo.parameters(), lr = 0.0001)
# Define a loss
criterion = CombinedLoss(nn.BCEWithLogitsLoss(),nn.BCEWithLogitsLoss(),nn.BCEWithLogitsLoss(),_lambda = 0.7)
n_params = 0

In [ ]:
# Train model
train(combo, (loaders_a, loaders_b), optimizer, criterion, epochs=7, dev=dev)

Epoch 1: Training Loss for combo = 0.6526,
Epoch 1: Training Accuracy for A = 0.7959,
Epoch 1: Training Accuracy for B = 0.7931,
Epoch 1: Training Accuracy for combo = 0.6075,
Epoch 1: Val Loss for combo = 1.7742,
Epoch 1: Val Accuracy for A = 0.5108,
Epoch 1: Val Accuracy for B = 0.5054,
Epoch 1: Val Accuracy for combo = 0.5081,
Epoch 1: Test Loss for combo = 1.7828,
Epoch 1: Test Accuracy for A = 0.5029,
Epoch 1: Test Accuracy for B = 0.5086,
Epoch 1: Test Accuracy for combo = 0.5058,


Epoch 2: Training Loss for combo = 0.3746,
Epoch 2: Training Accuracy for A = 0.9382,
Epoch 2: Training Accuracy for B = 0.9490,
Epoch 2: Training Accuracy for combo = 0.6350,
Epoch 2: Val Loss for combo = 2.6567,
Epoch 2: Val Accuracy for A = 0.5108,
Epoch 2: Val Accuracy for B = 0.5054,
Epoch 2: Val Accuracy for combo = 0.5081,
Epoch 2: Test Loss for combo = 2.6714,
Epoch 2: Test Accuracy for A = 0.5029,
Epoch 2: Test Accuracy for B = 0.5086,
Epoch 2: Test Accuracy for combo = 0.5058,


Epoch 3: Tra

learning rate = 0.001, Adam e lambda=0.2 e 7 epoche



In [ ]:
model1 = VGG16((1,32,32),batch_norm=True)
model2 = VGG16((1,32,32),batch_norm=True,classifier=model1.classifier)
combo = TiedVGG16(model1,model2,0, MAX)

In [ ]:
optimizer = torch.optim.Adam(combo.parameters(), lr = 0.001)
# Define a loss
criterion = CombinedLoss(nn.BCEWithLogitsLoss(),nn.BCEWithLogitsLoss(),nn.BCEWithLogitsLoss(),_lambda = 0.2)
n_params = 0

In [ ]:
# Train model
train(combo, (loaders_a, loaders_b), optimizer, criterion, epochs=7, dev=dev)

Epoch 1: Training Loss for combo = 0.9655,
Epoch 1: Training Accuracy for A = 0.5014,
Epoch 1: Training Accuracy for B = 0.5224,
Epoch 1: Training Accuracy for combo = 0.5623,
Epoch 1: Val Loss for combo = 0.7722,
Epoch 1: Val Accuracy for A = 0.4892,
Epoch 1: Val Accuracy for B = 0.4946,
Epoch 1: Val Accuracy for combo = 0.5081,
Epoch 1: Test Loss for combo = 0.7722,
Epoch 1: Test Accuracy for A = 0.4971,
Epoch 1: Test Accuracy for B = 0.4914,
Epoch 1: Test Accuracy for combo = 0.5058,


Epoch 2: Training Loss for combo = 0.6916,
Epoch 2: Training Accuracy for A = 0.5890,
Epoch 2: Training Accuracy for B = 0.6895,
Epoch 2: Training Accuracy for combo = 0.5944,
Epoch 2: Val Loss for combo = 1.2174,
Epoch 2: Val Accuracy for A = 0.5108,
Epoch 2: Val Accuracy for B = 0.5054,
Epoch 2: Val Accuracy for combo = 0.5081,
Epoch 2: Test Loss for combo = 1.2190,
Epoch 2: Test Accuracy for A = 0.5029,
Epoch 2: Test Accuracy for B = 0.5086,
Epoch 2: Test Accuracy for combo = 0.5058,


Epoch 3: Tra